## Multi-Modal RAG with Azure AI Content Understanding - In Action

![rag_in_action](./Assets/rag_in_action.png)

In [2]:
%pip install python-dotenv openai azure-identity


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\asridharan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### Setting up the Environment

In [1]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
import requests
import json

load_dotenv()

# Azure OpenAI Configurations
# Use AZURE_AI_ENDPOINT for OpenAI SDK calls (.openai.azure.com domain)
azure_ai_endpoint = os.getenv("AZURE_AI_ENDPOINT")
azure_ai_api_key = os.getenv("AZURE_AI_API_KEY")
embedding_model_name = os.getenv("EMBEDDING_MODEL_NAME")
chat_model_name = os.getenv("CHAT_MODEL_NAME")

# Azure Search Service Configurations
search_service_endpoint = os.getenv("SEARCH_SERVICE_ENDPOINT").strip().rstrip('/')
search_service_api_key = os.getenv("SEARCH_SERVICE_API_KEY")
search_service_index_name = os.getenv("SEARCH_SERVICE_INDEX_NAME")

# Authentication Mode
AUTH_MODE = os.getenv("AUTH_MODE", "entra").lower()  # "entra" or "key"

# Setup credential for Entra ID auth
credential = None
if AUTH_MODE == "entra":
    credential = DefaultAzureCredential()
    print("✅ Using Entra ID authentication")
else:
    print("🔑 Using API Key authentication")

print(f"Azure OpenAI Endpoint: {azure_ai_endpoint}")
print(f"Search Endpoint: {search_service_endpoint}")
print(f"Embedding Model: {embedding_model_name}")
print(f"Chat Model: {chat_model_name}")

✅ Using Entra ID authentication
Azure OpenAI Endpoint: https://demo-aisvcs-eus.openai.azure.com
Search Endpoint: https://arvsearch123.search.windows.net
Embedding Model: text-embedding-3-large
Chat Model: gpt-4o


### Creating Azure OpenAI Client

In [3]:
from openai import AzureOpenAI

# Use Entra ID authentication if AUTH_MODE is set to "entra"
if AUTH_MODE == "entra":
    from azure.identity import get_bearer_token_provider
    
    token_provider = get_bearer_token_provider(
        credential, 
        "https://cognitiveservices.azure.com/.default"
    )
    
    openai_client = AzureOpenAI(
        azure_ad_token_provider=token_provider,
        api_version="2024-02-15-preview",
        azure_endpoint=azure_ai_endpoint
    )
else:
    openai_client = AzureOpenAI(
        api_key=azure_ai_api_key,
        api_version="2024-02-15-preview",
        azure_endpoint=azure_ai_endpoint
    )

print("✅ Azure OpenAI client created")

✅ Azure OpenAI client created


### Creating a Function for Generating Vector Embeddings using Azure OpenAI

In [4]:
from openai import AzureOpenAI

def generate_embeddings(text, embedding_model_name):
    response = openai_client.embeddings.create(
        input=text,
        model=embedding_model_name
    )

    embeddings = response.model_dump()

    return embeddings["data"][0]["embedding"]

In [5]:
"""
Queries to try out:
1. tell me something about BMW Circularity Project
2. Tell me something about BMW's forwardism strategy
3. Tell me something about BMW's sustainable natural rubber initiative
4. Tell me through BMW's sustainability journey from 1973 to 2030
"""

user_query = "tell me something about BMW Circularity Project"
vectorised_user_query = generate_embeddings(user_query, embedding_model_name)
print(vectorised_user_query)

[0.00525951711460948, -0.025200506672263145, -0.011696778237819672, -0.027426928281784058, -0.04191480204463005, 0.003573567373678088, 0.021796341985464096, 0.032525110989809036, 0.00022889385581947863, 0.002161887241527438, 0.016319023445248604, -0.048755399882793427, 0.020408861339092255, -0.011559643782675266, 0.004727111663669348, -0.014802474528551102, -0.008937952108681202, 0.0026116082444787025, 0.06592143326997757, -0.03820409998297691, -0.024474499747157097, -0.046690311282873154, -0.06366274505853653, 0.03229924291372299, -0.05872589349746704, 0.0015346979489549994, 0.00717133516445756, -0.020731531083583832, -0.04549643397331238, -0.014826674945652485, 0.03021802194416523, 0.0330091156065464, -0.00413017300888896, 0.004041438456624746, -0.01559301558881998, -0.01865031197667122, 0.03355765342712402, -0.03513873741030693, 0.005360351409763098, 0.023651693016290665, -0.009300955571234226, 0.03092789463698864, -0.011220840737223625, 0.030992429703474045, -0.030572958290576935, 

In [6]:
context=[]

### Sending API call to the Search Index to Build Context

In [7]:
import requests
import json


url = f"{search_service_endpoint}/indexes/{search_service_index_name}/docs/search?api-version=2023-11-01"

# Set up headers based on auth mode
if AUTH_MODE == "entra":
    search_token = credential.get_token("https://search.azure.com/.default").token
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {search_token}"
    }
else:
    headers = {
        "Content-Type": "application/json",
        "api-key": search_service_api_key
    }
    
body = {
    "count": True,
    "select": "content_text",
    "vectorQueries": [
        {
            "vector": vectorised_user_query,
            "k": 10,
            "fields": "content_embedding",
            "kind": "vector"
        }
    ]
}
    
response = requests.post(url, headers=headers, data=json.dumps(body))
documents = response.json()['value']

for doc in documents:
    context.append(dict(
        {
            "chunk": doc['content_text'],
            "score": doc['@search.score']
            
        }
    ))
    
for doc in context:
    print(doc)

{'chunk': '{"about": "This is a JSON string representing a slice of PDF for RAG.", "pdf_source": "https://multimodalragsa2.blob.core.windows.net/ragdata/BMW_sustainable_natural_rubber.pdf", "page_number": 1, "page_text": "High-Level Commitment of the BMW Group\\nfor Sustainable Natural Rubber\\n\\nProtection of forests and other natural ecosystems is critical for maintaining biodiversity, combating climate\\nchange, and sustaining livelihoods. As part of our overall sustainability goals, the BMW Group is committed to\\neliminating deforestation and ecosystem conversion from our supply chains and to safeguarding human rights\\nacross all our operations and suppliers. Given that natural rubber is a known driver of deforestation, this\\ndocument outlines our commitment to sourcing sustainable natural rubber and is aligned with the Policy\\nFramework that was adopted in a September 2020 resolution by the General Assembly of the Global Platform\\nfor Sustainable Natural Rubber (GPSNR), of w

### Setting the System Prompt

In [8]:
system_prompt = """
You are a professional RAG-based assistant whose context comes exclusively from a database in Azure AI Search

Always follow these rules:

1. Answer strictly from the context provided. Do not invent, assume, or add details outside the given context.
2. If no relevant information is available in the context, politely say so.
3. Do not include any external links, citations, or references unless they are explicitly present in the context object.
4. Context format: The system will pass a Python list of objects, each containing:
   {
     "chunk": "the content (text, JSON, transcript, or description)",
     "score": "the relevancy score"
   }
   These are the top matches based on cosine similarity with the user query.
5. Style & tone:
   - Respond in a professional, natural way, as if conversing with a human.
   - Structure answers clearly and concisely.
   - Do not reveal that retrieval-augmented generation (RAG) is being used under the hood.
   - If the context contains a field such as 'url', 'video_url', or 'image_url', include it as a clickable hyperlink in your response, but do not fabricate or assume URLs.

Your role is to act like a knowledgeable human assistant who can reference the provided information smoothly and contextually, across any modality (text, image, video, or review description).
"""


### Calling GPT Engine for Summarisation

In [9]:
user_query = f""" The user query is: {user_query}
              the context is: {context} """

chat_completions_response = openai_client.chat.completions.create(
    model = chat_model_name,
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ],
    temperature=0.7
)

print(chat_completions_response.choices[0].message.content)

The BMW Circularity Project is referenced in the provided context through a video summary. You can access the related video [here](https://multimodalragsa2.blob.core.windows.net/ragdata/BMW_circularity.mp4). Unfortunately, specific details about the project are not included in the context, so I cannot provide more information at this time. Please let me know if you'd like assistance with anything else!


### Testing Multiple Queries Against Multi-Modal RAG

In [ ]:
import os
from datetime import datetime

def ask_rag(query):
    """Helper function to run a full RAG query and return the response."""
    # Generate embedding for the query
    query_embedding = generate_embeddings(query, embedding_model_name)
    
    # Search the index
    search_url = f"{search_service_endpoint}/indexes/{search_service_index_name}/docs/search?api-version=2023-11-01"
    
    if AUTH_MODE == "entra":
        token = credential.get_token("https://search.azure.com/.default").token
        search_headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {token}"
        }
    else:
        search_headers = {
            "Content-Type": "application/json",
            "api-key": search_service_api_key
        }
    
    search_body = {
        "count": True,
        "select": "content_text, document_title",
        "vectorQueries": [{
            "vector": query_embedding,
            "k": 5,
            "fields": "content_embedding",
            "kind": "vector"
        }]
    }
    
    search_response = requests.post(search_url, headers=search_headers, json=search_body)
    results = search_response.json().get('value', [])
    
    # Build context from results
    query_context = [{"chunk": doc['content_text'], "title": doc['document_title'], "score": doc['@search.score']} for doc in results]
    
    # Generate response
    full_query = f"The user query is: {query}\nthe context is: {query_context}"
    
    response = openai_client.chat.completions.create(
        model=chat_model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": full_query}
        ],
        temperature=0.7
    )
    
    return response.choices[0].message.content, query_context

# Test queries covering different modalities
test_queries = [
    "What is BMW's approach to circularity?",
    "Tell me about BMW's forwardism strategy",
    "What are BMW's sustainability initiatives for natural rubber?"
]

# Create output folder
output_folder = "rag_results"
os.makedirs(output_folder, exist_ok=True)

# Build markdown content
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
md_content = f"""# Multi-Modal RAG Test Results

**Generated:** {timestamp}

**Model:** {chat_model_name}  
**Embedding Model:** {embedding_model_name}  
**Search Index:** {search_service_index_name}

---

"""

print("=" * 80)
print("🧪 MULTI-MODAL RAG TEST RESULTS")
print("=" * 80)

for i, query in enumerate(test_queries, 1):
    print(f"\n📝 Query {i}: {query}")
    print("-" * 60)
    
    answer, ctx = ask_rag(query)
    
    print(f"\n🤖 Response:\n{answer}")
    print(f"\n📚 Sources used:")
    for c in ctx[:3]:  # Show top 3 sources
        print(f"   • {c['title']} (score: {c['score']:.4f})")
    print("\n" + "=" * 80)
    
    # Add to markdown content
    md_content += f"""## Query {i}: {query}

### Response

{answer}

### Sources Used

| Source | Relevance Score |
|--------|-----------------|
"""
    for c in ctx:
        md_content += f"| {c['title']} | {c['score']:.4f} |\n"
    
    md_content += "\n---\n\n"

# Save to markdown file
output_file = os.path.join(output_folder, f"rag_test_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md")
with open(output_file, "w", encoding="utf-8") as f:
    f.write(md_content)

print(f"\n✅ Results saved to: {output_file}")

🧪 MULTI-MODAL RAG TEST RESULTS

📝 Query 1: What is BMW's approach to circularity?
------------------------------------------------------------

🤖 Response:
BMW's approach to circularity is not explicitly detailed in the provided context. However, the company demonstrates a commitment to sustainability in its operations, as seen in its high-level commitments to sustainable natural rubber. This includes eliminating deforestation, protecting ecosystems, safeguarding human rights, and ensuring traceability and risk assessment in supply chains. They also engage in multi-stakeholder initiatives and promote compliance with international laws and guidelines.

For more insights, you might explore their specific strategies on circularity in the provided [video](https://multimodalragsa2.blob.core.windows.net/ragdata/BMW_circularity.mp4), which may contain detailed information on their circular economy initiatives.

📚 Sources used:
   • BMW Sustainable Natural Rubber PDF • Page 1 (score: 0.6282)
 